# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [7]:
#Your code here
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# load dataset
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
timestamp    8188 non-null object
id           8188 non-null int64
group        8188 non-null object
action       8188 non-null object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [9]:
df.loc[df['id'] == 363314]

,timestamp,id,group,action
3751,2016-11-17 15:51:56.634476,363314,control,view
3752,2016-11-17 15:52:30.776850,363314,control,click


In [10]:
# looks like the id will appear twice if they have more than one action
# How many clicks did we get?
df.loc[df['action'] == 'click']['id'].count()

1860

In [13]:
# check for data inconsistencies - - did anyone click who didn't view?
clicked_ids = df.loc[df['action'] == 'click']['id']
clicked_ids

8       349125
12      601714
15      487634
17      468601
21      555973
         ...  
8162    451198
8164    252195
8167    344770
8180    615849
8182    795585
Name: id, Length: 1860, dtype: int64

In [30]:
# go through each id and check if there is a corresponding entry for views
num_missing_views = 0
for clicked_id in clicked_ids:
    temp_df = df.loc[df['id'] == clicked_id]
    if len(temp_df) != 2:
        num_missing_views += 1
print(num_missing_views)

0


In [34]:
# there does not appear to be any click ids that correspond to ids with no views
# check for overlaps in control and experimental group
control_ids = df.loc[df['group'] == 'control']['id']
overlaps = []
for control_id in control_ids:
    temp_df = df.loc[(df['id'] == control_id) & (df['group'] == 'experiment')]
    if len(temp_df) > 0:
        overlaps.append(temp_df)

print(overlaps)

[]


In [35]:
# there are no overlaps between the control and experimental groups, we are good to move forward with our statistical test

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [36]:
#Your code here
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [37]:
df.columns

Index(['timestamp', 'id', 'group', 'action'], dtype='object')

In [38]:
df['count'] = 1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [43]:
# create control group
control = df.loc[df['group'] == 'control'].pivot(index='id', columns='action', values='count')
control = control.fillna(0)
control.head()

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [44]:
# create experimental group
experiment = df.loc[df['group'] == 'experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(0)
experiment.head()

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0


In [59]:
print(f"Sample Sizes:_____")
print(f"Control Group: {len(control)}")
print(f"Experiment Group: {len(experiment)}")
print(f"Clicks:_____")
print(f"Control Group: {control.click.sum()}")
print(f"Experiment Group: {experiment.click.sum()}")
print(f"Click Rate:_____")
print(f"Control Group: {round(control.click.mean(), 2)}")
print(f"Experiment Group: {round(experiment.click.mean(), 2)}")

Sample Sizes:_____
Control Group: 3332
Experiment Group: 2996
Clicks:_____
Control Group: 932.0
Experiment Group: 928.0
Click Rate:_____
Control Group: 0.28
Experiment Group: 0.31


In [62]:
pval = stats.ttest_ind(control.click, experiment.click, equal_var=False)[1] / 2
print(f"pval: {pval}")

pval: 0.004466402814337101


In [63]:
# Null Hypothesis: Click Rate of Experiment Group = Click Rate of Control Group
# Alternative Hypothesis: Click Rate of Experiment Group > Click Rate of Control Group
# Using an alpha value of 0.05, we end up with a pvalue of 0.004, well lower than our alpha value
# We can reject the null hypothesis with 95% confidence in favor of the alternative hypothesis

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [69]:
#Your code here
control_rate = control['click'].mean()
expected_clicks = control_rate * len(experiment)
print(expected_clicks)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [73]:
# Your code here
actual_clicks = experiment['click'].sum()
n = len(experiment)
p = control_rate
var = n * p * (1 - p)
std = np.sqrt(var)
std

24.568547907005815

In [74]:
standard_deviations = (actual_clicks - expected_clicks) / std
standard_deviations

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [75]:
#Your code here
pval = stats.norm.sf(standard_deviations)
pval

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

This roughly matches the result of the previous statistical test at an alpha value of 0.05.  Both analyses reject the null hypothesis in favor of the null hypothesis that click rate is higher in the experiment group.  However, the second method produces "more statistically significant results"

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.